In [ ]:
from Algorithm import Gaussian_Process
from Algorithm import LA_Bayessian_Quadrature
from Algorithm import Mesh
from Algorithm import Kernels
import Target_Function as tf

import numpy as np


In [ ]:
opt_spy  = tf.Option('call', 0, 600, 0.1672, 0.15,  1, 100)   # SPDR S&P 500 ETF  (广义市场基准)
opt_aapl = tf.Option('call', 1, 200, 0.2613, 0.49,  1, 100)   # Apple
opt_tsla = tf.Option('call', 2, 350, 0.6398, 0.57,  1, 100)   # Tesla
opt_nvda = tf.Option('call', 3, 145, 0.3482, 0.24,  1, 100)   # NVIDIA
opt_amzn = tf.Option('call', 4, 210, 0.2902, 0.24,  1, 100)   # Amazon
opt_meta = tf.Option('call', 5, 700, 0.2924, 0.24,  1, 100)   # Meta
opt_msft = tf.Option('call', 6, 485, 0.1817, 0.15,  1, 100)   # Microsoft
opt_goog = tf.Option('call', 7, 165, 0.3226, 0.32,  1, 100)   # Google
opt_amd  = tf.Option('call', 8, 130, 0.4065, 0.32,  1, 100)   # AMD
opt_mstr = tf.Option('call', 9, 370, 0.5098, 0.49,  1, 100)   # MicroStrategy

def LABQ(D):
    options = [opt_spy, opt_aapl, opt_tsla, opt_nvda, opt_amzn,
               opt_meta, opt_msft, opt_goog, opt_amd, opt_mstr]
    portfolio = tf.OptionPortfolio(
        0.04,50,900,
        options[:D]
    )
    D = len(portfolio.options)
    func = portfolio.target_function
    xx = np.linspace(0., 1., 3)
    grids = np.meshgrid(*([xx] * D),)
    X = np.vstack([g.ravel() for g in grids]).T
    Y = np.array([func(x) for x in X])
    beta = np.concatenate([
    np.array([-0.5]),
    np.random.uniform(-2, 0, size=11 * D)
    ])
    mesh1 = Mesh.get_points_Sobol(X,_)
    GP = Gaussian_Process.GaussianProcess(Kernels.K_lin_mat2_10,beta,X,Y,mesh = mesh1)
    a, b, time, uncertainty = LA_Bayessian_Quadrature.ABC_D(
        func,
        GP,
        Mesh.get_points_Sobol,
        n = min(30,max(49-6*D,3)),
        lambd = [30, 1],
        point_mesh = mesh1,
        sobol = min(7,12-D)
    )
    return time,uncertainty

result = []
for D in range(2,11):
    """
    2D --> 31 Steps, Sobol: 2^7
    3D --> 31 Steps, Sobol: 2^7
    4D --> 26 Steps, Sobol: 2^7
    5D --> 20 Steps, Sobol: 2^7
    6D --> 14 Steps, Sobol: 2^6
    7D --> 8 Steps, Sobol: 2^5
    8D --> 4 Steps, Sobol: 2^4
    9D --> 4 Steps, Sobol: 2^3
    10D --> 4 Steps, Sobol: 2^2

    """
    time, uncertainty = LABQ(D)
    result.append([D,time,uncertainty])
print(result)